# <center>Large-scale methods: column generation</center>
### <center>Alfred Galichon (NYU & Sciences Po)</center>
## <center>'math+econ+code' masterclass series</center>
#### <center>With python code examples</center>
© 2018–2023 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274 are acknowledged, as well as inputs from contributors listed [here](http://www.math-econ-code.org/team).

**If you reuse material from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass series. https://www.math-econ-code.org/


### References

* Dantzig, G.B. and Wolfe, P. (1960). "Decomposition Principle for Linear Programs". *Operations Research*. 8: 101–111.

* Luenberger, D. and Ye, Y. (2008). *Linear and Nonlinear Optimization*. Springer.

In [1]:
import numpy as np
from mec.lp import Tableau
import gurobipy as grb
from sympy import symbols

In [2]:
class Pivoter():
    def __init__(self,B_i_b,x_b):
        self.nbb = B_i_b.shape[0]
        self.B_i_b = B_i_b
        self.x_b = x_b
    
    def determine_departing(self,Nent_i):
        z_b = np.linalg.solve(self.B_i_b,Nent_i) 
        thedic = {b: self.x_b[b] / z_b[b] for b in range(self.nbb) if z_b[b]>0}
        bdep = min(thedic, key = thedic.get)
        epsilon = thedic[bdep]
        return bdep , epsilon
    
    def update(self,bdep,epsilon,Nent_i):
        self.x_b = self.x_b - epsilon * np.linalg.solve(self.B_i_b,Nent_i) 
        self.x_b[bdep] = epsilon
        self.B_i_b[:,bdep] = Nent_i

Consider the problem<br>
$\max_{x \geq 0} c^\top x$<br>
$Ax = d$

Assume that variables are given by\
$B x_B + N x_N = d$.

The objective function can be written as:\
$c^\top_B B^{-1}d +(c^\top_N- c^\top_B B^{-1} N )x_N$

We can thus define the dual simplex variable:

$y = (B^{-1} )^\top c_B $

so that

$
A^{\top }y-c=
\begin{pmatrix}
B^{\top }y-c_{B} \\ 
N^{\top }y-c_{N}
\end{pmatrix}
=
\begin{pmatrix}
0_B \\ 
N^{\top }y-c_{N}
\end{pmatrix}
$

and thus complementary slackness<br>
$x^\top = x^\top_B 0_B + x^\top_N (A^{\top }y-c)=0$<br>
is satisfied, as $x_N = 0$.

Yet dual feasibility is not attained unless $t_N \geq 0$, where<br> 
$t_N = N^\top y - c_N$<br>
is the dual slack variable


In [68]:
class MatrixTableau(Pivoter):
    def __init__(self,B_i_b,N_i_n, d_i,c_n, c_b=None  ,names_basic = None, names_nonbasic = None):
        Pivoter.__init__(self,B_i_b,np.linalg.solve(B_i_b, d_i))
        self.N_i_n = N_i_n
        self.d_i = d_i
        if c_b is None:
            self.c_b = np.zeros(self.nbb)
        else:
            self.c_b = c_b
        self.c_n = c_n
        self.nbn = N_i_n.shape[1]
        if names_basic is None:
            self.names_basic = ['s_'+str(b) for b in range(self.nbb)]
        else:
            self.names_basic = names_basic
        if names_nonbasic is None:
            self.names_nonbasic = ['x_'+str(n) for n in range(self.nbn)]
        else:
            self.names_nonbasic = names_nonbasic
            
                
    def display(self):
        expr_i = self.d_i - np.linalg.solve(self.B_i_b, self.N_i_n) @ symbols(self.names_nonbasic)
        for i in range(self.nbb) :
            print(self.names_basic[i]+'='+str(expr_i[i] ) )
        print('obj=',self.x_b.dot(self.c_b))

    def update(self,bdep,nent,epsilon):
        Nent_i,Bdep_i = self.N_i_n[:,nent].copy(),self.B_i_b [:, bdep].copy()
        Pivoter.update(self,bdep,epsilon,Nent_i)
        self.N_i_n [ :, nent ]  = Bdep_i
        self.c_b[ bdep ] , self.c_n [ nent ] = self.c_n [ nent ] , self.c_b[ bdep ]
        self.names_basic[ bdep ] , self.names_nonbasic[ nent ] =  self.names_nonbasic[ nent ] , self.names_basic[ bdep ]
        
    def determine_departing(self,nent):
        return Pivoter.determine_departing(self,self.N_i_n[:,nent])
        
    def determine_entering(self):
        t_n = self.N_i_n.T @ np.linalg.solve(self.B_i_b.T,self.c_b) - self.c_n  
        entering_candidates = [ (n,self.names_nonbasic[n]) for n in range(self.nbn) if (t_n[n] < 0 ) ]
        if (entering_candidates):
            return min(entering_candidates, key = lambda couple: couple[1])[0]
        else:
            return None
        
    def gurobi_solve(self ,verbose=0):
        m = grb.Model()
        gx_b = m.addMVar(self.nbb)
        gx_n = m.addMVar(self.nbn)
        m.Params.OutputFlag = verbose
        gconstr_i = m.addConstr(self.B_i_b @ gx_b + self.N_i_n @ gx_n == self.d_i)
        m.setObjective(gx_b @ self.c_b+ gx_n @ self.c_n, sense=grb.GRB.MAXIMIZE)
        m.optimize()
        return(m.objVal)
    
    def solve(self,verbose = 0):
        niter = 0
        while True:
            niter += 1
            nent = self.determine_entering()
            if nent is None:
                if verbose > 0:
                    print('\nConverged in '+str(niter)+' steps. Solution:')
                    self.display()
                return
            else:
                bdep,eps = self.determine_departing(nent)
                if verbose>0:
                    print('iter='+str(niter)+': entering='+self.names_nonbasic[nent]+'; departing='+self.names_basic[bdep]+'; obj='+str(self.x_b.dot(self.c_b)))
                self.update( bdep , nent ,eps)

In [69]:
example_tableau = MatrixTableau(np.eye(2),
                          np.array([[2, 1], [1, 2]]),
                          np.array([2,2]),
                          np.array([1,1]))

print(example_tableau.names_basic)
print(example_tableau.names_nonbasic)
print(example_tableau.gurobi_solve())
example_tableau.display()
example_tableau.solve(1)

['s_0', 's_1']
['x_0', 'x_1']
1.3333333333333335
s_0=-2.0*x_0 - 1.0*x_1 + 2
s_1=-1.0*x_0 - 2.0*x_1 + 2
obj= 0.0
iter=1: entering=x_0; departing=s_0; obj=0.0
iter=2: entering=x_1; departing=s_1; obj=1.0

Converged in 3 steps. Solution:
x_0=-0.666666666666667*s_0 + 0.333333333333333*s_1 + 2
x_1=0.333333333333333*s_0 - 0.666666666666667*s_1 + 2
obj= 1.3333333333333335


In [72]:
import pandas as pd
thepath = 'https://raw.githubusercontent.com/math-econ-code/mec_optim_2021-01/master/data_mec_optim/lp_stigler-diet/'
filename = 'StiglerData1939.txt'
thedata = pd.read_csv(thepath + filename, sep='\t')
thedata = thedata.dropna(how = 'all')
commodities = (thedata['Commodity'].values)[:-1]
allowance = thedata.iloc[-1, 4:].fillna(0).transpose()
N_i_j = thedata.iloc[:-1, 4:].fillna(0).transpose().values
N_i_j = N_i_j[:5,:7]
nbi, nbj = N_i_j.shape
c_j = np.ones(nbj)
d_i = np.array(allowance)[:nbi]

diet_example = MatrixTableau( np.eye(nbj),N_i_j.T,c_j,d_i , None, ['s_' + str(j) for j in range(nbj)],
                      ['pi_' + str(i) for i in range(nbi)])

In [73]:
diet_example.solve(1)


iter=1: entering=pi_0; departing=s_0; obj=0.0
iter=2: entering=pi_2; departing=s_2; obj=0.06711409395973153
iter=3: entering=pi_4; departing=s_4; obj=0.10243839504580053
iter=4: entering=pi_1; departing=pi_0; obj=0.13316571742073996
iter=5: entering=pi_3; departing=pi_1; obj=0.14192722677918596
iter=6: entering=s_0; departing=pi_3; obj=0.17198226331405836

Converged in 7 steps. Solution:
s_0=-43.0611111111111*pi_0 - 1358.63888888889*pi_1 - 340.694444444444*pi_3 + 0.138888888888889*s_2 + 1.0
s_1=-11.0263888888889*pi_0 - 399.673611111111*pi_1 - 45.4930555555556*pi_3 + 0.0486111111111111*s_2 + 1.0
pi_2=-0.819444444444444*pi_0 - 26.1805555555556*pi_1 - 12.1527777777778*pi_3 - 0.0694444444444444*s_2 + 1.0
s_3=-11.3180555555556*pi_0 - 249.381944444444*pi_1 - 54.7847222222222*pi_3 + 0.00694444444444444*s_2 + 1.0
pi_4=-1.11996583962603*pi_0 - 27.5887720244516*pi_1 - 2.53528407047825*pi_3 + 0.00382056814095649*s_2 - 0.0323624595469256*s_4 + 1.0
s_5=-27.9444444444444*pi_0 - 659.055555555556*pi_1